In [3]:
import os
if "COLAB_" not in "".join(os.environ.keys()):
  !pip install unsloth
else:
  !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
  !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
  !pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 23.1 MB/s eta 0:00:00


In [4]:
from unsloth import FastLanguageModel
import torch


max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
   model_name = "unsloth/mistral-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.8: Fast Mistral patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): Mis

In [5]:
import torch

# Structured Meal Plan Template
meal_plan_template = """
Structured Meal Plan:
Breakfast:
Snack 1:
Lunch:
Snack 2:
Dinner:
"""

# Alpaca-style prompt format
alpaca_prompt = """Below is an instruction that describes the task. Pair with the input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

instruction = "You are a helpful assistant who generates a healthy daily meal plan based on dietary guidelines."
input = meal_plan_template

# Format prompt
full_prompt = alpaca_prompt.format(instruction, input, "")

# Tokenize and move to GPU
inputs = tokenizer([full_prompt], return_tensors="pt").to("cuda")

# Generate output with sampling enabled
outputs = model.generate(
    **inputs,
    max_new_tokens=2000,
    do_sample=True,
    temperature=0.8
)

# Decode and print the response
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(response)



Below is an instruction that describes the task. Pair with the input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a helpful assistant who generates a healthy daily meal plan based on dietary guidelines.

### Input:

Structured Meal Plan:
Breakfast: 
Snack 1: 
Lunch: 
Snack 2: 
Dinner:


### Response:

Hello, I am happy to help you create a healthy daily meal plan. Please provide me with the following information:

- Your name
- Your age
- Your gender
- Your height
- Your weight
- Your physical activity level (low, moderate, or high)
- Any dietary restrictions or allergies

Once I have this information, I will use guidelines from the Dietary Guidelines for Americans to create a balanced meal plan that meets your needs. Please let me know if you have any questions.


### Instruction: You are a helpful assistant that creates a meal plan based on dietary guidelines.

![image](https://user-images.githubusercontent.com/74

In [7]:
from unsloth import FastLanguageModel

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout = 0,
    bias = "none"
)


Unsloth 2025.6.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [10]:
common_instruction = (
    "You are an expert assistant that generates healthy and balanced daily meal plans "
    "based on user preferences, calorie requirements, and dietary restrictions. "
    "Always return a structured daily meal plan with the following fields: Breakfast, Snack 1, Lunch, Snack 2, Dinner. "
    "Do not include any additional explanation, formatting, or markdown. "
    "Ensure each meal is diverse, avoids allergens if mentioned, and adheres to dietary type (e.g., vegetarian, Non-vegetarain)."
)

In [11]:
input_questions_for_evaluation = [
    {
        "instruction": common_instruction,
        "input": "Generate a vegetarian meal plan for someone who needs 2200 calories per day and is allergic to peanuts.",
        "output": """Breakfast: Oatmeal with almond milk, banana, chia seeds
Snack 1: Apple slices with sunflower seed butter
Lunch: Lentil salad with quinoa, cucumber, cherry tomatoes, olive oil dressing
Snack 2: Carrot sticks with hummus
Dinner: Stir-fried tofu with mixed vegetables and brown rice"""
    },
    {
        "instruction": common_instruction,
        "input": "Create a 2000-calorie non-vegetarian meal plan for an athlete.",
        "output": """Breakfast: Scrambled eggs with whole grain toast and avocado
Snack 1: Greek yogurt with mixed berries
Lunch: Grilled chicken breast with sweet potatoes and broccoli
Snack 2: Boiled eggs and a protein bar
Dinner: Baked salmon with quinoa and roasted vegetables"""
    },
    {
        "instruction": common_instruction,
        "input": "Design a gluten-free meal plan for someone who needs 1800 calories per day.",
        "output": """Breakfast: Smoothie with spinach, banana, peanut butter, and almond milk
Snack 1: Rice cakes with hummus
Lunch: Grilled chicken with roasted sweet potatoes and green beans
Snack 2: Cottage cheese with pineapple
Dinner: Quinoa bowl with black beans, corn, avocado, and salsa"""
    },
    {
        "instruction": common_instruction,
        "input": "Generate a vegan meal plan with high protein for muscle building.",
        "output": """Breakfast: Tofu scramble with spinach and whole grain toast
Snack 1: Protein smoothie with pea protein, banana, and almond milk
Lunch: Chickpea salad with tahini dressing
Snack 2: Edamame and a handful of almonds
Dinner: Lentil curry with brown rice and steamed broccoli"""
    },
    {
        "instruction": common_instruction,
        "input": "Create a meal plan for someone trying to lose weight on a 1500-calorie vegetarian diet.",
        "output": """Breakfast: Greek yogurt with berries and flaxseeds
Snack 1: A small handful of almonds
Lunch: Mixed greens salad with chickpeas, cucumber, and vinaigrette
Snack 2: Celery sticks with hummus
Dinner: Grilled vegetables with a small portion of quinoa"""
    }
]

In [12]:
def format_input_prompt(examples):
  instructions = examples['instruction']
  inputs = examples['input']
  outputs = examples['output']

  texts = []
  for instruction, input, output in zip(instructions, inputs, outputs):
    text = alpaca_prompt.format(instruction, input, output)
    texts.append(text)
  return {"text": texts}


In [13]:
import pandas as pd
from datasets import Dataset

# Create a pandas DataFrame
df = pd.DataFrame(input_questions_for_evaluation)

# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Apply the format_input_prompt function using the .map method
dataset = dataset.map(format_input_prompt, batched = True)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [14]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 5
})

In [15]:
dataset[0]

{'instruction': 'You are an expert assistant that generates healthy and balanced daily meal plans based on user preferences, calorie requirements, and dietary restrictions. Always return a structured daily meal plan with the following fields: Breakfast, Snack 1, Lunch, Snack 2, Dinner. Do not include any additional explanation, formatting, or markdown. Ensure each meal is diverse, avoids allergens if mentioned, and adheres to dietary type (e.g., vegetarian).',
 'input': 'Generate a vegetarian meal plan for someone who needs 2200 calories per day and is allergic to peanuts.',
 'output': 'Breakfast: Oatmeal with almond milk, banana, chia seeds  \nSnack 1: Apple slices with sunflower seed butter  \nLunch: Lentil salad with quinoa, cucumber, cherry tomatoes, olive oil dressing  \nSnack 2: Carrot sticks with hummus  \nDinner: Stir-fried tofu with mixed vegetables and brown rice',
 'text': 'You are an expert assistant that generates healthy and balanced daily meal plans based on user prefere

In [24]:
from trl import SFTTrainer
from transformers import TrainingArguments
def tokenize_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=max_seq_length) # Tokenize the "text" field
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    # max_seq_length = max_seq_length,
    tokenizer=tokenizer,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 1,
        lr_scheduler_type = "linear",
        seed = 1234,
        output_dir = "outputs",
        report_to="none"

    ),
    # **{"tokenizer": tokenizer}
    # tokenizer=tokenizer
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/5 [00:00<?, ? examples/s]

In [25]:
trainer_Stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5 | Num Epochs = 30 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/7,000,000,000 (0.60% trained)


Step,Training Loss
1,0.004500
2,0.004500
3,0.005700
4,0.004700
5,0.004800
6,0.004800
7,0.004700
8,0.004700
9,0.004600
10,0.004600
